In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [2]:
diag = pd.read_csv("../data/diagnoses_icd.csv", dtype = str)

In [3]:
diag.head(5)

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9
1,15734973,20475282,2,V0251,9
2,15734973,20475282,5,V270,9
3,15734973,20475282,1,64891,9
4,15734973,20475282,4,66481,9


In [4]:
d=diag['icd_code']
# print(diag['icd_code'])
d.head(5)
d.to_csv("../icd_code.csv")

In [5]:
print(diag.shape[0])
print(diag.icd_code.nunique())
print(diag.subject_id.nunique())

5280351
27170
255106


In [6]:
group_icd_codes = True

In [7]:
if group_icd_codes:
    diag["icd_code"] = diag["icd_code"].apply(lambda x: str(x)[:8])

In [8]:
#plenty of people diagnosed Gestational Diabetes
print(diag[diag["icd_code"] == "O244"].subject_id.nunique())

0


In [9]:
diag["icd_code"].head()

0     2825
1    V0251
2     V270
3    64891
4    66481
Name: icd_code, dtype: object

In [10]:
#use first 3 digits to group diagonisis codes to avoid sparsity
diag_clean = diag.loc[:,["subject_id", "icd_code"]]
diag_count = diag_clean.groupby("icd_code").count()

In [11]:
print(diag_count.shape)
print(diag_count[diag_count.subject_id < 5].shape)

(27170, 1)
(11792, 1)


In [12]:
diag_clean.head(5)

,subject_id,icd_code
0,15734973,2825
1,15734973,V0251
2,15734973,V270
3,15734973,64891
4,15734973,66481


In [13]:
admission = pd.read_csv("../data/admissions.csv", dtype = str)
admission = admission.loc[:,["subject_id", "hadm_id", "admittime"]]

In [14]:
diag_w_time = diag.merge(admission, how = "inner", on=["subject_id", "hadm_id"])
diag_w_time.rename(columns = {"admittime": "time", "icd_code":"events"}, inplace = True )

In [15]:
diag_w_time.shape

(5280347, 6)

In [16]:
diag_w_time.head()

,subject_id,hadm_id,seq_num,events,icd_version,time
0,15734973,20475282,3,2825,9,2112-02-02 07:53:00
1,15734973,20475282,2,V0251,9,2112-02-02 07:53:00
2,15734973,20475282,5,V270,9,2112-02-02 07:53:00
3,15734973,20475282,1,64891,9,2112-02-02 07:53:00
4,15734973,20475282,4,66481,9,2112-02-02 07:53:00


In [17]:
diag_clean = diag_w_time.loc[:,["subject_id", "events", "time"]]

In [18]:
#filter out all empty or null diag codes
diag_clean["events"] = diag_clean.events.str.strip()
diag_clean = diag_clean[~diag_clean.events.isnull()]
diag_clean = diag_clean[diag_clean.events != ""]
diag_clean.shape

(5280347, 3)

In [19]:
#make sure no empty/null events
print(diag_clean[diag_clean.events.apply(lambda x: len(x.strip())) == 0].shape[0])
print(diag_clean[diag_clean.events.isnull()].shape[0])

0
0


In [20]:
#diagnosis data with time
diag_clean.head()

,subject_id,events,time
0,15734973,2825,2112-02-02 07:53:00
1,15734973,V0251,2112-02-02 07:53:00
2,15734973,V270,2112-02-02 07:53:00
3,15734973,64891,2112-02-02 07:53:00
4,15734973,66481,2112-02-02 07:53:00


In [21]:
pres = pd.read_csv("../data/prescriptions.csv", dtype = str)

In [22]:
#again extremely skewed distribution, might need to group rare codes 
pres_clean = pres.loc[:, ["subject_id", "starttime", "drug"]]
drug_ct = pres_clean.groupby("drug").count()["subject_id"]
drug_ct.describe()

count     10254.000000
mean       1658.674859
std       17580.626662
min           1.000000
25%           1.000000
50%           2.000000
75%          14.000000
max      637105.000000
Name: subject_id, dtype: float64

In [23]:
pres_clean.shape[0]

17008053

In [24]:
pres_clean.rename(columns = {"starttime":"time", "drug":"events"},inplace = True)

In [25]:
pres_clean.head()

,subject_id,time,events
0,17868682,2160-01-07 08:00:00,BuPROPion (Sustained Release)
1,17868682,2160-01-07 08:00:00,BuPROPion (Sustained Release)
2,17868682,2160-01-07 10:00:00,Aspirin
3,17868682,2160-01-07 14:00:00,1/2 NS
4,17868682,2160-01-07 15:00:00,Pneumococcal Vac Polyvalent


In [26]:
#delete all null/empty observations
pres_clean["events"] = pres_clean.events.str.strip()
pres_clean["events"] = pres_clean.events.str.lower()

pres_clean = pres_clean[~pres_clean.events.isnull()]
pres_clean = pres_clean[pres_clean.events != ""]

#make sure no empty/null events
print(pres_clean[pres_clean.events.apply(lambda x: len(x.strip())) == 0].shape[0])
print(pres_clean[pres_clean.events.isnull()].shape[0])
print(pres_clean.shape[0])

0
0
17008052


In [27]:
pres_clean.head()

,subject_id,time,events
0,17868682,2160-01-07 08:00:00,bupropion (sustained release)
1,17868682,2160-01-07 08:00:00,bupropion (sustained release)
2,17868682,2160-01-07 10:00:00,aspirin
3,17868682,2160-01-07 14:00:00,1/2 ns
4,17868682,2160-01-07 15:00:00,pneumococcal vac polyvalent


In [28]:
proc = pd.read_csv("../data/procedures_icd.csv", dtype = str)

In [29]:
proc_w_time = proc.merge(admission, how = "inner", on=["subject_id", "hadm_id"])
proc_w_time.rename(columns = {"admittime": "time", "icd_code":"events"}, inplace = True )

In [30]:
proc_w_time.head(5)

,subject_id,hadm_id,seq_num,chartdate,events,icd_version,time
0,10287061,27485182,1,2124-08-22,7569,9,2124-08-21 16:23:00
1,13135573,22286210,3,2187-05-30,7309,9,2187-05-29 16:04:00
2,13135573,22286210,1,2187-05-30,7569,9,2187-05-29 16:04:00
3,13135573,22286210,2,2187-05-30,734,9,2187-05-29 16:04:00
4,12646116,24154012,2,2115-01-13,734,9,2115-01-12 17:23:00


In [31]:
proc_w_time.shape[0]

779625

In [32]:
proc_clean = proc_w_time.loc[:,["subject_id", "time","events"]]
proc_clean.head()

,subject_id,time,events
0,10287061,2124-08-21 16:23:00,7569
1,13135573,2187-05-29 16:04:00,7309
2,13135573,2187-05-29 16:04:00,7569
3,13135573,2187-05-29 16:04:00,734
4,12646116,2115-01-12 17:23:00,734


In [33]:
# no empty/null value in procesdure data set
print(proc_clean[proc_clean.events.isnull()].shape[0])
print(proc_clean[proc_clean.events.apply(lambda x: len(x.strip())) == 0].shape[0])

0
0


In [34]:
if group_icd_codes:
    proc_clean["events"] = proc_clean["events"].apply(lambda x: str(x)[:3])

In [35]:
proc_clean.time.min()

'2105-10-04 17:26:00'

In [36]:
proc_events = pd.read_csv("../data/procedureevents.csv", dtype = str)

In [37]:
proc_events.head()

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,value,valueuom,location,...,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_date,originalamount,originalrate
0,15693895,21203492,30500789,2147-06-05 11:25:00,2147-06-05 11:26:00,2147-06-06 11:25:00,225399,1,None,NaN,...,80,NaN,NaN,0,0,0,FinishedRunning,NaN,1,0
1,15693895,21203492,30500789,2147-06-05 22:00:00,2147-06-05 22:01:00,2147-06-06 11:25:00,224385,1,None,NaN,...,80,NaN,NaN,0,0,0,FinishedRunning,NaN,1,0
2,12481680,26876606,30863119,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393,min,NaN,...,123,NaN,NaN,1,0,0,FinishedRunning,NaN,393,1
3,12481680,26876606,30863119,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393,min,NaN,...,123,NaN,NaN,1,0,0,FinishedRunning,NaN,393,1
4,15614172,27424463,33484414,2153-07-29 18:13:00,2153-07-31 01:25:00,2153-07-31 01:26:00,224277,1872,min,RL Post Forearm,...,71.6,NaN,NaN,1,1,0,FinishedRunning,NaN,1872,1


In [38]:
proc_events[proc_events.ordercategorydescription == "Task"].ordercategoryname.value_counts()

Procedures               164354
Imaging                  120577
Intubation/Extubation     32441
Communication             31740
Significant Events        27114
CRRT Filter Change          251
Name: ordercategoryname, dtype: int64

In [39]:
proc_events.ordercategorydescription.value_counts()

Task                 376477
ContinuousProcess    354770
Name: ordercategorydescription, dtype: int64

In [40]:
proc_clean_MV = proc_events.loc[proc_events.ordercategorydescription == "Task", ["subject_id", "starttime", "itemid"]]
item_id = pd.read_csv("../data/d_items.csv", dtype = str)
item_id = item_id.loc[:,["itemid", "label"]]
proc_clean_MV = proc_clean_MV.merge(item_id, on = "itemid")

In [41]:
proc_clean_MV.rename(columns = {"starttime" : "time","label": "events"}, inplace = True)
proc_clean_MV = proc_clean_MV.drop(["itemid"], axis = 1)

In [42]:
print(proc_clean_MV[proc_clean.events.isnull()].shape[0])
print(proc_clean_MV[proc_clean.events.apply(lambda x: len(x.strip()) == 0)].shape[0])

proc_clean_MV["events"] = proc_clean_MV.events.str.strip()
proc_clean_MV["events"] = proc_clean_MV.events.str.lower()

C:\Users\84755\AppData\Local\Temp/ipykernel_4340/1880869949.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(proc_clean_MV[proc_clean.events.isnull()].shape[0])
C:\Users\84755\AppData\Local\Temp/ipykernel_4340/1880869949.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(proc_clean_MV[proc_clean.events.apply(lambda x: len(x.strip()) == 0)].shape[0])


0
0


In [43]:
proc_clean_MV.time.min()

'2110-01-11 12:40:00'

In [53]:
use_mv = False

if use_mv:
    all_events_data = pd.concat([diag_clean,proc_clean_MV, pres_clean], axis = 0)
else:
    all_events_data = pd.concat([diag_clean,proc_clean, pres_clean], axis = 0)

In [54]:
all_events_data.shape[0]

23068024

In [55]:
all_events_data.head()

,subject_id,events,time
0,15734973,2825,2112-02-02 07:53:00
1,15734973,V0251,2112-02-02 07:53:00
2,15734973,V270,2112-02-02 07:53:00
3,15734973,64891,2112-02-02 07:53:00
4,15734973,66481,2112-02-02 07:53:00


In [56]:
from sklearn.utils import shuffle
#randomly shuffle the data so it breaks the sequence order with diag and procedure with same subject id and time
all_events_data = shuffle(all_events_data).reset_index(drop=True)

In [57]:
#turn medication, diag, procedure events into integer ids (by their alphabetical order)
all_events = all_events_data.events.unique()
all_events.sort()
index = np.arange(1,len(all_events)+1)
events_lookup = pd.DataFrame({"events":all_events, "eve_index":index})

In [58]:
if use_mv:
    events_lookup.to_csv("../cleaned_data/events_id_mv.csv")
else:
    events_lookup.to_csv("../cleaned_data/events_id.csv")

In [59]:
#merge all_events_data with event_index
all_events_data = all_events_data.merge(events_lookup, on = "events")

In [60]:
all_events_data = all_events_data.sort_values(by = ["subject_id", "time"]).reset_index(drop=True)
all_events_data.head()

,subject_id,events,time,eve_index
0,10000019,V3000,2129-05-21 19:16:00,26446
1,10000019,640,2129-05-21 19:16:00,5346
2,10000019,V290,2129-05-21 19:16:00,26425
3,10000019,V053,2129-05-21 19:16:00,26072
4,10000019,V502,2129-05-21 19:16:00,26637


In [61]:
if use_mv:
    all_events_data.to_csv("../cleaned_data/all_events_data_mv.csv")
else:
    all_events_data.to_csv("../cleaned_data/all_events_data.csv")